Итак, мы увидели в действии обычные, разреженные и шумоподавляющие автокодировщики. А теперь, чтобы диалектически завершить эту главу синтезом двух ее основных тем, давайте вернемся к сверточным сетям и попробуем реализовать сверточный автокодировщик.  

Как он будет работать? Об автокодировщиках мы уже многое знаем: суть их в том, чтобы сначала построить некое внутреннее представление на внутреннем слое нейронов, а потом «развернуть» его обратно, реконструировать вход на выходе. Мы до сих пор все время рассматривали сверточные слои, которые берут на вход некую «картинку» и применяют к ней сверточные фильтры, получая представление
в виде сначала совсем локальных, а потом все более и более глобальных признаков. Это отлично подходит для построения первой половины автокодировщика, от входа до внутреннего представления.  

Но как же потом разворачивать? В случае полносвязной сети мы просто строили такую же, симметричную архитектуру для декодирования; нам было не так уж важно, какие будут размеры у матрицы весов: 784 х 128 или наоборот. Но со свертками не все так очевидно.  

Чтобы ответить на этот вопрос, давайте введем операцию деконволюции, или транспонированной свертки. Каждый фильтр в сверточном слое можно представить как операцию, сжимающую область k х k в одно число; с другой стороны, мы можем определить и обратную операцию — развернуть одно число в матрицу k х k. Нужно только аккуратно учесть шаг свертки и дополнение нулями, но по сути мы можем просто транспонировать сверточный тензор и получить деконволюцию.  

Разумеется, в TensorFlow такая операция уже реализована. Как всегда, начинаем с импорта библиотек и загрузки набора данных:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
batch_size, learning_rate = 64, 0.01

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Веса автокодировщика задаются как для обычной сверточной сети.

In [2]:
ae_weights = {"conv": tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1)),
              "b_hidden": tf.Variable(tf.truncated_normal([4], stddev=0.1)),
              "deconv": tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1)), 
              "b_visible": tf.Variable(tf.truncated_normal([1], stddev=0.1))
             }

Напомним, что первые две размерности тензора задают размер фильтра, а третья и четвертая, соответственно, число фильтров в текущем слое и в следующем. На первый взгляд, удивительно: почему декодирующая часть имеет те же размерности, что и кодирующая? Разве не нужно транспонировать матрицы? Причина здесь чисто техническая: TensorFlow сама транспонирует веса и применит их так, как надо; единственное, что нам для этого будет нужно, — это дополнительно передать в TensorFlow тензор с размерами результирующего слоя.

In [5]:
input_shape = tf.stack([batch_size, 28, 28, 1])

Поскольку мы реализуем автокодировщик, то выходной слой должен иметь такой же размер, как и входной. Теперь мы готовы определить все необходимые для обучения тензоры:

In [6]:
ae_input = tf.placeholder(tf.float32, [batch_size, 784])
images = tf.reshape(ae_input, [-1, 28, 28, 1])
hidden_logits = tf.nn.conv2d(ae_input, ae_weights["conv"], 
                             strides=[1, 2, 2, 1], padding="SAME") + ae_weights["b_hidden"]
hidden = tf.nn.sigmoid(hidden_logits)

visible_logits = tf.nn.conv2d_transpose(hidden, ae_weights["deconv"], 
                                       input_shape, strides=[1, 2, 2, 1],
                                       padding="SAME") + ae_weights["b_visible"]
visible = tf.nn.sigmoid(visible_logits)

ValueError: Shape must be rank 4 but is rank 2 for 'Conv2D' (op: 'Conv2D') with input shapes: [64,784], [5,5,1,4].